# Set Up Matrix Input

In [1]:
import numpy as np
import time

# Row and Col size of matrix being bulitiplied
row_size = 4
col_size = 64

# Generate random input matrix
# A = np.random.rand(row_size,col_size)
# B = np.random.rand(col_size,row_size)
A = np.full((row_size,col_size),1)
B = np.full((col_size,row_size),1)


py_out = np.zeros((row_size,row_size))
py_out = np.matmul(A,B)

# print(A,B,py_out)

# Converters Float to Fixed Point and Vice Versa

In [2]:
def float_to_fp(value, total_bits=16, frac_bits=8, output_format='hex'):
    # Step 1: Check if the value is negative
    if value < 0:
        is_negative = True
        value = -value  # Work with the positive equivalent for conversion
    else:
        is_negative = False

    # Step 2: Multiply the float by 2^frac_bits to scale it
    scaled_value = round(value * (2 ** frac_bits))
    
    # Step 3: Convert the scaled value to a binary string
    # Format the binary string to the required total bits length
    binary_value = bin(scaled_value)[2:].zfill(total_bits)

    # Step 4: Handle the negative case
    if is_negative:
        # Perform two's complement conversion for negative numbers
        # Subtract the binary value from the max value for two's complement
        max_value = (1 << total_bits)  # This is 2^total_bits
        binary_value = bin(max_value - scaled_value)[2:].zfill(total_bits)

    # Step 5: Convert the binary value to hexadecimal
    hex_value = hex(int(binary_value, 2))[2:].zfill(total_bits // 4)  # Convert to hex and remove '0x'

    # Step 6: Format output based on the `output_format` argument
    if output_format == 'binary':
        return binary_value
    elif output_format == 'hex':
        return hex_value
    elif output_format == 'both':
        return (f"{total_bits}'b{binary_value}", f"{total_bits}'h{hex_value}")
    else:
        raise ValueError("Invalid output format. Choose 'binary', 'hex', or 'both'.")
    
def fp_to_float(value,total_bits=16, frac_bits=8, input_format='hex'):
    if input_format == 'binary': fp_val = int(value,2)
    elif input_format == 'hex': fp_val = int(value,16)
    else: raise ValueError("Invalid input format. Must be 'binary' or 'hex'")
    # Calculate the scaling factor: 2^frac_width
    scaling_factor = 2 ** frac_bits
    # Perform the division to convert to floating point
    float_value = fp_val / scaling_factor
    
    return float_value

# Example usage:
value = 1.75  # Example float number

num1 = float_to_fp(value)
inv_num1 = fp_to_float(str(num1))
print(num1,inv_num1)

01c0 1.75


# Create File for RTL Testing

In [3]:
def wr_input(matrix,filename):
    cnt = 0
    f = open(filename,'w')
    for i in range(8):
        for j in range(15):
            if j == 0:
                elem0 = float_to_fp(matrix[i][0])
                elem1 = float_to_fp(0)
                elem2 = float_to_fp(0)
                elem3 = float_to_fp(0)
            elif j == 1:
                elem0 = float_to_fp(matrix[i][0])
                elem1 = float_to_fp(matrix[i][1])
                elem2 = float_to_fp(0)
                elem3 = float_to_fp(0)
            elif j == 2:
                elem0 = float_to_fp(matrix[i][0])
                elem1 = float_to_fp(matrix[i][1])
                elem2 = float_to_fp(matrix[i][2])
                elem3 = float_to_fp(0)
            elif j == 8:
                elem0 = float_to_fp(0)
                elem1 = float_to_fp(matrix[i][1])
                elem2 = float_to_fp(matrix[i][2])
                elem3 = float_to_fp(matrix[i][3])
            elif j == 9:
                elem0 = float_to_fp(0)
                elem1 = float_to_fp(0)
                elem2 = float_to_fp(matrix[i][2])
                elem3 = float_to_fp(matrix[i][3])
            elif j == 10:
                elem0 = float_to_fp(0)
                elem1 = float_to_fp(0)
                elem2 = float_to_fp(0)
                elem3 = float_to_fp(matrix[i][3])
            elif j > 10:
                elem0 = float_to_fp(0)
                elem1 = float_to_fp(0)
                elem2 = float_to_fp(0)
                elem3 = float_to_fp(0)
            else:
                elem0 = float_to_fp(matrix[i][0])
                elem1 = float_to_fp(matrix[i][1])
                elem2 = float_to_fp(matrix[i][2])
                elem3 = float_to_fp(matrix[i][3])
            
            f.write(f'{elem0}{elem1}{elem2}{elem3}\n')
    f.close()

wr_input(A.transpose(),"..\\data_west.txt")
wr_input(B,"..\\data_north.txt")

# Run Testbench

In [4]:
%%cmd
cd ..
iverilog.exe -c .\block_4x32.txt
vvp .\a.out

Microsoft Windows [Version 10.0.22631.4602]
(c) Microsoft Corporation. All rights reserved.

c:\Users\fauza\Documents\Univ\Sem_7\EL5035_VLSI_for_Digital_System_Processing\project\block_based_matrix_multiplication\python>cd ..

c:\Users\fauza\Documents\Univ\Sem_7\EL5035_VLSI_for_Digital_System_Processing\project\block_based_matrix_multiplication>iverilog.exe -c .\block_4x32.txt

c:\Users\fauza\Documents\Univ\Sem_7\EL5035_VLSI_for_Digital_System_Processing\project\block_based_matrix_multiplication>vvp .\a.out
VCD info: dumpfile wave.vcd opened for output.
VCD warning: array word block_4x32_tb.data_north[0] will conflict with an escaped identifier.
VCD warning: array word block_4x32_tb.data_west[0] will conflict with an escaped identifier.
VCD warning: array word block_4x32_tb.data_north[1] will conflict with an escaped identifier.
VCD warning: array word block_4x32_tb.data_west[1] will conflict with an escaped identifier.
VCD warning: array word block_4x32_tb.data_north[2] will conflict 

# Read RTL Output File and Check Error

In [5]:
rtl_out = np.zeros((row_size,row_size))

with open('..\\out_val.txt', 'r') as f:
    # Iterate through each line in the file
    for i, line in enumerate(f):
        # Strip newline characters or extra spaces
        line = line.strip()
        
        # Parse the line into chunks of 4 characters
        for j in range(row_size):
            start_index = j * 4
            end_index = start_index + 4
            temp = line[start_index:end_index]
            print(temp)
            rtl_out[i][j] = fp_to_float(temp)


err = py_out - rtl_out
print(f"{py_out}\n{rtl_out}\n{err}")

4000
3800
3800
3800
3800
4000
4000
4000
3800
4000
4000
4000
3800
4000
4000
4000
[[64 64 64 64]
 [64 64 64 64]
 [64 64 64 64]
 [64 64 64 64]]
[[64. 56. 56. 56.]
 [56. 64. 64. 64.]
 [56. 64. 64. 64.]
 [56. 64. 64. 64.]]
[[0. 8. 8. 8.]
 [8. 0. 0. 0.]
 [8. 0. 0. 0.]
 [8. 0. 0. 0.]]
